# dumpのテスト　

http://archive.ics.uci.edu/ml/index.php　
から、落として来た、Physicochemical Properties of Protein Tertiary Structure Data Set
(タンパク質の三次構造データセットの物理化学的性質)
というものを使って見る。
特徴量は9こで、
F1 - 総表面積。
F2 - 非極性露出領域。
F3 - 露出した非極性残留物の部分面積。
F4 - 残渣の露出した非極性部分の小面積。
F5 - 分子量加重露出面積。
F6 - 残留物の標準露出面積からの平均偏差。
F7 - ユークリッド距離。
F8 - 二次構造ペナルティ。
F9 - 空間分布制約（N、K値）。

求めたいのは、
RMSD-残渣のサイズ。(Size of the residue.)(値は0~21)
欠損値はなし、F8は整数値で他は少数値。

In [2]:

import pandas as pd
import numpy as np

df = pd.read_csv('CASP.csv', header=0, )

#一列目がy


X = df.iloc[:,1:]
y = df.iloc[:,[0]]

#check the shape
print('----------------------------------------------------------------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)
print('y shape: (%i,%i)' %y.shape)
print('----------------------------------------------------------------------------------------')
print(X.dtypes)
print('----------------------------------------------------------------------------------------')
print('Check the null count of the target variable: %i' % y.isnull().sum())
print('----------------------------------------------------------------------------------------')

print('----------------------------------------------------------------------------------------')

print(y.head())
X.head()

----------------------------------------------------------------------------------------
Raw shape: (45730,10)
X shape: (45730,9)
y shape: (45730,1)
----------------------------------------------------------------------------------------
F1    float64
F2    float64
F3    float64
F4    float64
F5    float64
F6    float64
F7    float64
F8      int64
F9    float64
dtype: object
----------------------------------------------------------------------------------------
Check the null count of the target variable: 0
----------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------
     RMSD
0  17.284
1   6.021
2   9.275
3  15.851
4   7.962


,F1,F2,F3,F4,F5,F6,F7,F8,F9
0,13558.30,4305.35,0.31754,162.1730,1.872791e+06,215.3590,4287.87,102,27.0302
1,6191.96,1623.16,0.26213,53.3894,8.034467e+05,87.2024,3328.91,39,38.5468
2,7725.98,1726.28,0.22343,67.2887,1.075648e+06,81.7913,2981.04,29,38.8119
3,8424.58,2368.25,0.28111,67.8325,1.210472e+06,109.4390,3248.22,70,39.0651
4,7460.84,1736.94,0.23280,52.4123,1.021020e+06,94.5234,2814.42,41,39.9147


In [3]:
# import basice apis
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from skgarden.mondrian.ensemble import MondrianForestRegressor

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20, random_state=1)

# data Standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scl = scaler.transform(X_train)
X_test_scl = scaler.transform(X_test)

#set learning machine
knr = KNeighborsRegressor()
lir = LinearRegression()
rid = Ridge()
rfr = RandomForestRegressor(random_state=42)
gbr = GradientBoostingRegressor(random_state=42)
mfr = MondrianForestRegressor()
mfr_p = MondrianForestRegressor()

dumpして読み込めるかができるかのテストを行う。
fit→dump→読み込み→partil_fit→・・・は一番下

In [4]:
#とりあえず全部fitさせてみる

apis = [knr, lir, rid, rfr, gbr, mfr]
for api in apis:
    api.fit(X_train_scl, y_train.as_matrix().ravel())
    
#また、Mondrianのpartial_fit(batchだが)もやってみる
mfr_p.partial_fit(X_train_scl, y_train.as_matrix().ravel())
print('end')

end


In [5]:
#dumpして再読み込みのテストとりあえずRandomForestでできるか

from sklearn.externals import joblib

print(rfr.predict(X_test_scl))
joblib.dump(rfr, 'rfr.pkl')

[ 7.984   2.8659  8.4859 ... 14.1199  1.4481 13.3344]


['rfr.pkl']

In [6]:
rfr_re = joblib.load('rfr.pkl')
print(rfr_re.predict(X_test_scl))

[ 7.984   2.8659  8.4859 ... 14.1199  1.4481 13.3344]


In [7]:
#Mondrainはどうか
print(mfr.predict(X_test_scl))
print(mfr_p.predict(X_test_scl))

joblib.dump(mfr, 'mfr.pkl')
joblib.dump(mfr_p, 'mfr_p.pkl')

[ 5.28076758  2.9557816  10.25956306 ... 10.35358653  1.85576398
  9.72776637]
[ 6.7380075   2.12987145 10.11324434 ... 12.37635136  1.73983341
 12.33106923]


['mfr_p.pkl']

In [7]:
#実行するとエラーが出て、karnel is died

#mfr_re = joblib.load('mfr.pkl')
#mfr_p_re = joblib.load('mfr_p.pkl')

#mfr_re.predict(X_test_scl)
#mfr_p_re.predict(X_test_scl)

In [8]:
#別のdumpでも試す。

import pickle
with open('rfr.pickle', 'wb') as f:
    pickle.dump(rfr, f)

In [9]:
with open('rfr.pickle', 'rb') as f:
    rfr_re2 = pickle.load(f)
rfr_re2.predict(X_test_scl)

#確かに保存されている。

array([ 7.984 ,  2.8659,  8.4859, ..., 14.1199,  1.4481, 13.3344])

In [10]:
import pickle
with open('mfr.pickle', 'wb') as f:
    pickle.dump(mfr, f)
with open('mfr_p.pickle', 'wb') as f:
    pickle.dump(mfr_p, f)

In [11]:
with open('mfr.pickle', 'rb') as f:
    mfr_re2 = pickle.load(f)
with open('mfr_p.pickle', 'rb') as f:
    mfr_p_re2 = pickle.load(f)
mfr_re2.predict(X_test_scl)
mfr_p_re2.predict(X_test_scl)

array([ 6.7380075 ,  2.12987145, 10.11324434, ..., 12.37635136,
        1.73983341, 12.33106923])

joblibでできなかったが、なぜかpickleの方ではできた。
partial_fitで保存されていくか、確認する。
ちょっと順番でやっていく

In [12]:
from skgarden.mondrian.ensemble import MondrianForestRegressor
from sklearn.metrics import r2_score

mfr_x = MondrianForestRegressor()
mfr_x.partial_fit(X_train_scl[:5000,:], y_train[:5000].as_matrix().ravel())
r2_score(y_test.as_matrix().ravel(), mfr_x.predict(X_test_scl))

0.5000674166985366

In [13]:
import pickle
with open('mfr_x1.pickle', 'wb') as f:
    pickle.dump(mfr_x, f)
with open('mfr_x1.pickle', 'rb') as f:
    mfr_x1 = pickle.load(f)
r2_score(y_test.as_matrix().ravel(), mfr_x1.predict(X_test_scl))

#ロードできてると思われる。

0.5000674166985366

In [14]:
mfr_x.partial_fit(X_train_scl[5000:10000,:], y_train[5000:10000].as_matrix().ravel())
r2_score(y_test.as_matrix().ravel(), mfr_x.predict(X_test_scl))

0.5374112284544452

In [15]:
import pickle
with open('mfr_x2.pickle', 'wb') as f:
    pickle.dump(mfr_x, f)
with open('mfr_x2.pickle', 'rb') as f:
    mfr_x2 = pickle.load(f)
r2_score(y_test.as_matrix().ravel(), mfr_x2.predict(X_test_scl))
#ロードできてると思われる。

0.5374112284544452

In [16]:
mfr_x.partial_fit(X_train_scl[10000:15000,:], y_train[10000:15000].as_matrix().ravel())
r2_score(y_test.as_matrix().ravel(), mfr_x.predict(X_test_scl))


0.5713539032633672

In [17]:
import pickle
with open('mfr_x3.pickle', 'wb') as f:
    pickle.dump(mfr_x, f)
with open('mfr_x3.pickle', 'rb') as f:
    mfr_x3 = pickle.load(f)
r2_score(y_test.as_matrix().ravel(), mfr_x3.predict(X_test_scl))
#ロードできてると思われる。

0.5713539032633672

In [18]:
#全部
mfr_x.partial_fit(X_train_scl[15000:,:], y_train[15000:].as_matrix().ravel())
r2_score(y_test.as_matrix().ravel(), mfr_x.predict(X_test_scl))

0.6341124918534603

In [19]:
import pickle
with open('mfr_x4.pickle', 'wb') as f:
    pickle.dump(mfr_x, f)
with open('mfr_x4.pickle', 'rb') as f:
    mfr_x4 = pickle.load(f)
r2_score(y_test.as_matrix().ravel(), mfr_x4.predict(X_test_scl))
#ロードできてると思われる。

0.6341124918534603

In [20]:
#一気にやってみる。
mfr_xx = MondrianForestRegressor()
mfr_xx.partial_fit(X_train_scl, y_train.as_matrix().ravel())
r2_score(y_test.as_matrix().ravel(), mfr_xx.predict(X_test_scl))

0.6275164362884542

In [21]:
#全部もう一回ロードしてみる。
with open('mfr_x1.pickle', 'rb') as f:
    mfr_test1 = pickle.load(f)
with open('mfr_x2.pickle', 'rb') as f:
    mfr_test2 = pickle.load(f)
with open('mfr_x3.pickle', 'rb') as f:
    mfr_test3 = pickle.load(f)
with open('mfr_x4.pickle', 'rb') as f:
    mfr_test4 = pickle.load(f)

print(r2_score(y_test.as_matrix().ravel(), mfr_test1.predict(X_test_scl)))
print(r2_score(y_test.as_matrix().ravel(), mfr_test2.predict(X_test_scl)))
print(r2_score(y_test.as_matrix().ravel(), mfr_test3.predict(X_test_scl)))
print(r2_score(y_test.as_matrix().ravel(), mfr_test4.predict(X_test_scl)))


0.49950114319175987
0.5359827095399881
0.5710401102593126
0.6341124918534603


やっぱり、pickleではできる気がする。
次に、保存→読み込み
partial_fitしてpredict
再度保存ができるかをチェック

In [22]:
#再定義して学習
mfr_xx = MondrianForestRegressor()
mfr_xx.partial_fit(X_train_scl[:5000,:], y_train[:5000].as_matrix().ravel())
print(r2_score(y_test.as_matrix().ravel(), mfr_xx.predict(X_test_scl)))

#保存する
import pickle
with open('mfr_xx1.pickle', 'wb') as f:
    pickle.dump(mfr_xx, f)


0.49204630894836754


In [23]:
#開く
with open('mfr_xx1.pickle', 'rb') as f:
    mfr_xx1 = pickle.load(f)

#再度fit
mfr_xx1.partial_fit(X_train_scl[5000:10000,:], y_train[5000:10000].as_matrix().ravel())
print(r2_score(y_test.as_matrix().ravel(), mfr_xx1.predict(X_test_scl)))

#再度保存
import pickle
with open('mfr_xx2.pickle', 'wb') as f:
    pickle.dump(mfr_xx1, f)

0.5339412592335394


In [24]:
#再度読み込み
with open('mfr_xx2.pickle', 'rb') as f:
    mfr_xx2 = pickle.load(f)
#再度fit
mfr_xx2.partial_fit(X_train_scl[10000:15000,:], y_train[10000:15000].as_matrix().ravel())
print(r2_score(y_test.as_matrix().ravel(), mfr_xx2.predict(X_test_scl)))

#再度保存
import pickle
with open('mfr_xx3.pickle', 'wb') as f:
    pickle.dump(mfr_xx2, f)


0.5662198014286459


In [25]:
#再度読み込み
with open('mfr_xx3.pickle', 'rb') as f:
    mfr_xx3 = pickle.load(f)
#再度fit
mfr_xx3.partial_fit(X_train_scl[15000:,:], y_train[15000:].as_matrix().ravel())
print(r2_score(y_test.as_matrix().ravel(), mfr_xx3.predict(X_test_scl)))

#再度保存
import pickle
with open('mfr_xx4.pickle', 'wb') as f:
    pickle.dump(mfr_xx3, f)


0.6337515588366657


In [26]:
#再度読み込み
with open('mfr_xx4.pickle', 'rb') as f:
    mfr_xx4 = pickle.load(f)
#最後にpredict
print(r2_score(y_test.as_matrix().ravel(), mfr_xx4.predict(X_test_scl)))



0.6337515588366657
